## section 1.2

In [2]:
from numpy import arange,array,ones,linalg, zeros
import numpy as np
from pylab import plot,show

from gurobipy import *
w=[1,1]
n=6
z=[4,7,1,3,9,2]
m = Model("calculL") 
Ni= np.arange(n)
Nk = np.arange(n)

b= m.addVars(Ni ,Nk,name="B",vtype=GRB.CONTINUOUS, lb=0)
r =m.addVars(Nk,name= "R",vtype=GRB.CONTINUOUS)
m.update()
obj = LinExpr();
obj =0
obj = quicksum( (( (k+1) * r[k]) - (quicksum(b[t,k] for t in Ni))) for k in Ni)

# definition de l'objectif
m.setObjective(obj,GRB.MAXIMIZE)

for i in Ni:
    for k in Ni:

        m.addConstr(r[k] -b [i,k] <= z[i])


m.optimize()
valsr = { k : v.X for k,v in r.items() }
valsb ={ k : v.X for k,v in b.items() }
#print(valsb)
d={}
for k in Ni:
    d[k]=(( (k+1) * valsr[k]) - (quicksum(valsb[t,k] for t in Ni))).getValue()
print(d)

Using license file C:\Users\moha1\gurobi.lic
Academic license - for non-commercial use only - expires 2023-11-01
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 36 rows, 42 columns and 72 nonzeros
Model fingerprint: 0xc673f159
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+00]
Presolve time: 0.01s
Presolved: 36 rows, 42 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1000000e+31   3.600000e+31   2.100000e+01      0s
      41    6.3000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 41 iterations and 0.01 seconds
Optimal objective  6.300000000e+01
{0: 1.0, 1: 3.0, 2: 6.0, 3: 10.0, 4: 17.0, 5: 26.0}


## section 1.4

In [30]:
from numpy import arange,array,ones,linalg, zeros
import numpy as np
from pylab import plot,show

from gurobipy import *

w=[1,1]
n=2

m = Model("projtest") 

Ni= np.arange(n)
Nx = np.arange(5)
z1=[[5,6,4,8,1],[3,8,6,2,5]]
z2=[3,8,6,2,5]


b= m.addVars(Ni ,Ni,name="B",vtype=GRB.CONTINUOUS, lb=0)
r =m.addVars(Ni,name= "R",vtype=GRB.CONTINUOUS)
x =m.addVars(Nx,name= "X",vtype=GRB.BINARY)

m.update()
obj = LinExpr();
obj =0



obj = quicksum( w[k] * (( (k+1) * r[k]) - (quicksum(b[t,k] for t in Ni))) for k in Ni)

# definition de l'objectif
m.setObjective(obj,GRB.MAXIMIZE)

m.addConstr(quicksum(x[t] for t in Nx)==3)
for i in Ni:
    for k in Ni:

        m.addConstr(r[k] -b [i,k] <= quicksum(x[t]*z1[i][t] for t in Nx))


m.optimize()
valsx = { k : v.X for k,v in x.items() if v.X ==1}

print(valsx)
res={}
for i in Ni:
  res[i] = quicksum(z1[i][k] for k in valsx).getValue()
print(res)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 5 rows, 11 columns and 33 nonzeros
Model fingerprint: 0xe36aaadd
Variable types: 6 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective 45.0000000
Presolve time: 0.00s
Presolved: 5 rows, 11 columns, 29 nonzeros
Variable types: 6 continuous, 5 integer (5 binary)

Root relaxation: objective 5.000000e+01, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      50.0000000   50.00000  0.00%     -    0s

Explored 0 nodes (7 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution

## section 2

In [4]:
def get_wk(w:list)->list:
    wk = w.copy()
    for i in range(len(wk)-1):
        wk[i] = w[i] -w[i+1]
    return wk

w=[3,2,1]
print(get_wk(w))

[1, 1, 1]


In [40]:
import numpy as np
def solve(n,p,w,u):

    i=0

    m = Model(f"projex{i}") 

    #pour les zij
    Ni= np.arange(n)
    Nj= np.arange(p)


    z= m.addVars(Ni ,Nj,name="Z",vtype=GRB.BINARY)
    b= m.addVars(Ni ,Ni,name="B",vtype=GRB.CONTINUOUS, lb=0)
    r =m.addVars(Ni,name= "R",vtype=GRB.CONTINUOUS)

    m.update()
    

    for j in Nj:
        m.addConstr(quicksum(z[i,j]for i in Ni)<=1)
        
    for i in Ni:
        for k in Ni:

            m.addConstr(r[k] -b [i,k] <= quicksum(z[i,p]*u[i,p] for p in Nj))
    # Resolution
    #m.addConstr(quicksum(z[i,j]*u[0,j]for j in Nj for i in Ni)<=900)

    obj = LinExpr();

    obj = quicksum( w[k] * (( (k+1) * r[k]) - (quicksum(b[t,k] for t in Ni))) for k in Ni)

    # definition de l'objectif
    m.setObjective(obj,GRB.MAXIMIZE)
    m.optimize()
    vals = { k : v.X for k,v in z.items() }
    d={}
    for i in range(u.shape[0]):
        d[i] = quicksum(u[i,j]*vals[i,j] for j in range(u.shape[1])).getValue()
    #print(vals)
    #print(d)
u=np.array([[325,225,210,115,75,50],[325,225,210,115,75,50],[325,225,210,115,75,50]])
n=3
p=6
solve(n,p,get_wk([3,2,1]),u)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 15 rows, 30 columns and 90 nonzeros
Model fingerprint: 0xac57ca72
Variable types: 12 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolved: 12 rows, 26 columns, 66 nonzeros
Found heuristic solution: objective 1000.0000000
Variable types: 8 continuous, 18 integer (18 binary)

Root relaxation: objective 2.000000e+03, 16 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2000.00000    0    4 1000.00000 2000.00000   100%     -    0s
H    0     0      

In [ ]:
import matplotlib.pyplot as plt
import time
def graph1(nbre_instances):
    nl=[2,5,10,15]
    #pl=[]
    #for i in range(len(nl)):
    #    pl.append(nl[i]*5)
    #liste_n_temps = []
    yAG_temps =[]
    for n in nl:
        p=5*n

        
        temps_AG=0
        for i in range(nbre_instances):
            u=np.random.randint(1,10, size=(n,p))
            var=random.sample(range(1,p*3), p)
            var.sort(reverse=True)
            w=get_wk(var)
            start_time = time.time()
            solve(n,p,w,u)
            temps_AG += time.time() - start_time
        yAG_temps.append(temps_AG/nbre_instances)
    

    titre = "Evolution du temps moyen de résolution en fonction de n et p= 5*n"
    plt.title(titre)
    plt.xlabel("n")
    plt.ylabel("Temps moyen en secondes")	
    #titre = "popsize = "+str(liste_popsize[i])
    print(f"size of yAGtemps{len(yAG_temps)} ,, size of nl {len(nl)}")
    plt.plot(nl, yAG_temps)
    plt.legend()
    plt.grid()
    plt.savefig("testfull.png")
    plt.show()

#graph1(10)

# section 3

In [41]:
from numpy import arange,array,ones,linalg, zeros
import numpy as np
from pylab import plot,show
import random

from gurobipy import *

def solve2(n,p,w,u,c,budget):
    
    m = Model("projtest") 

    #x = []
    #pour les zij

    Ni= np.arange(n)
    Nx = np.arange(p)
    



    b= m.addVars(Ni ,Ni,name="B",vtype=GRB.CONTINUOUS, lb=0)
    r =m.addVars(Ni,name= "R",vtype=GRB.CONTINUOUS)
    x =m.addVars(Nx,name= "X",vtype=GRB.BINARY)

    m.update()
    

    #m.addConstr(quicksum(x[t] for t in Nx)==n)
    for i in Ni:
        for k in Ni:

            m.addConstr(r[k] -b [i,k] <= quicksum(x[t]*u[i][t] for t in Nx))
    m.addConstr(quicksum(x[t]*c[t]for t in Nx)<=budget)
    obj = LinExpr();
    obj =0



    obj = quicksum( w[k] * (( (k+1) * r[k]) - (quicksum(b[t,k] for t in Ni))) for k in Ni)

    # definition de l'objectif
    m.setObjective(obj,GRB.MAXIMIZE)

    m.optimize()
    vals = { k : v.X for k,v in x.items() }
    d={}
    for i in range(u.shape[0]):
        d[i] = quicksum(u[i,j]*vals[j] for j in range(u.shape[1])).getValue()
    print(vals)
    print(d)
    
n=2
p=4
u=np.array([[19,6,17,2],[2,11,4,18]])
w=get_wk([2,1])
c=[40,50,60,50]
budget= np.sum(c)/2
solve2(n,p,w,u,c,budget)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 5 rows, 10 columns and 28 nonzeros
Model fingerprint: 0x6b7d2548
Variable types: 6 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 5 rows, 10 columns, 28 nonzeros
Variable types: 6 continuous, 4 integer (4 binary)

Root relaxation: objective 6.660000e+01, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   66.60000    0    1   -0.00000   66.60000      -     -    0s
H    0     0                      48.0000000   66.60000  38.7%     -    0s
H    0     0                      61.0

In [ ]:
import matplotlib.pyplot as plt
import time
def graph2(nbre_instances):
    nl= [2,5,10]
    pl= [5,10,15,20]
    liste_n_temps = []
    for p in pl:
        yAG_temps = []
        #yAG_nb_tentatives = []

        for n in nl:
            if n>p:
                yAG_temps.append(0)
                continue
            temps_AG = 0
            #nb_tentatives_AG = 0
            for i in range(nbre_instances):
                u=np.random.randint(1,10, size=(n,p))
                var=random.sample(range(1,p*3), p)
                var.sort(reverse=True)
                w=get_wk(var)
                c=np.random.randint(1,100, size=(p))
                budget= np.sum(c)/2
                start_time = time.time()
                solve2(n,p,w,u,c,budget)
                temps_AG += time.time() - start_time
            yAG_temps.append(temps_AG/nbre_instances)
        liste_n_temps.append(yAG_temps)
    titre = "Evolution du temps moyen de résolution en fonction de n et p"
    plt.title(titre)
    plt.xlabel("n")
    plt.ylabel("Temps moyen en secondes")
    for i in range(len(liste_n_temps)):
        titre = "p = "+str(pl[i])
        plt.plot(nl, liste_n_temps[i], label=titre)
    plt.legend()
    plt.grid()
    plt.savefig("fullTest2.png")
    plt.show()

#graph2(10)

# section 4

In [47]:
#4.1

import numpy as np
from gurobipy import *
N=7 #number of nodes
M= 10000 # a very large number

S1= np.array([[0,5,10,2,M,M,M],
              [M,0,4,1,4,M,M],
              [M,M,0,M,3,1,M],
              [M,M,1,0,M,3,M],
              [M,M,M,M,0,M,1],
              [M,M,M,M,M,0,1],
              [M,M,M,M,M,M,0]])
S2=np.array([[0,3,4,6,M,M,M],
              [M,0,2,3,6,M,M],
              [M,M,0,M,1,2,M],
              [M,M,4,0,M,5,M],
              [M,M,M,M,0,M,1],
              [M,M,M,M,M,0,1],
              [M,M,M,M,M,M,0]])

util=(S1,S2)
w=[0.8,0.1]
s=0
t=6
Ni= np.arange(N)
Nk=np.arange(2)
m = Model("projexee") 
#Nj= np.arange(p)
x= m.addVars(Ni ,Ni,name="X",vtype=GRB.BINARY)
b= m.addVars(Nk,Nk,name="B",vtype=GRB.CONTINUOUS, lb=0)
r =m.addVars(Nk,name= "R",vtype=GRB.CONTINUOUS,lb= -GRB.INFINITY)

m.update()
obj = LinExpr();

obj = quicksum( w[k] * (( (k+1) * r[k]) + (quicksum(b[t,k] for t in Nk))) for k in Nk)

m.setObjective(obj,GRB.MINIMIZE)
for i in Nk:
  for k in Nk:
    m.addConstr(r[k] + b [i,k] >= quicksum(x[h,j]*util[i][h,j] for h in Ni for j in Ni))

m.addConstr(quicksum(x[s,k] for k in Ni) - quicksum(x[i,s] for i in Ni) == 1) # source node flux = 1
m.addConstr(quicksum(x[t,k] for k in Ni) - quicksum(x[i,t] for i in Ni) == -1) # source node flux = -1

for j in Ni: # intermediate node flux = 0
    if j != s and j != t:
        m.addConstr(quicksum(x[j,k] for k in Ni) - quicksum(x[i,j] for i in Ni) == 0)
        
m.optimize()
vals = { k : v.X for k,v in x.items() if v.X==1 }

for i,j in vals:
  print(f"{i} and {j}")
res={}
for h in Nk:
  res[h] = quicksum(util[h][i,j] for i,j in vals).getValue()
print(res)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 11 rows, 55 columns and 260 nonzeros
Model fingerprint: 0x0c8c9160
Variable types: 6 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e-01, 8e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 10000.000000
Presolve removed 1 rows and 9 columns
Presolve time: 0.00s
Presolved: 10 rows, 46 columns, 195 nonzeros
Variable types: 3 continuous, 43 integer (42 binary)

Root relaxation: objective 9.272727e+00, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.27273    0    4 10000.0000    9.27273   100%     -    0s
H    0     0                      11.5000000    9.27273  19.4%  

In [45]:
#4.2
def solve3(w,util):
  N=7 #number of nodes
  s=0
  t=6
  Ni= np.arange(N)
  Nk=np.arange(2)
  m = Model("projex") 
  #Nj= np.arange(p)
  x= m.addVars(Ni ,Ni,name="X",vtype=GRB.BINARY)
  b= m.addVars(Nk,Nk,name="B",vtype=GRB.CONTINUOUS, lb=0)
  r =m.addVars(Nk,name= "R",vtype=GRB.CONTINUOUS,lb= -GRB.INFINITY,ub=GRB.INFINITY)

  m.update()
 
  for i in Nk:
    for k in Nk:
      m.addConstr(r[k] + b [i,k] >= quicksum(x[h,j]*util[i][h,j] for h in Ni for j in Ni))

  m.addConstr(quicksum(x[s,k] for k in Ni) - quicksum(x[i,s] for i in Ni) == 1) # source node flux = 1
  m.addConstr(quicksum(x[t,k] for k in Ni) - quicksum(x[i,t] for i in Ni) == -1) # source node flux = -1

  for j in Ni: # intermediate node flux = 0
      if j != s and j != t:
          m.addConstr(quicksum(x[j,k] for k in Ni) - quicksum(x[i,j] for i in Ni) == 0)

  obj = LinExpr();

  obj = quicksum( w[k] * (( (k+1) * r[k]) + (quicksum(b[t,k] for t in Nk))) for k in Nk)

  m.setObjective(obj,GRB.MINIMIZE)

  m.optimize()
  vals = { k : v.X for k,v in x.items() if v.X==1 }

  res={}
  for h in Nk:
    res[h] = quicksum(util[h][i,j] for i,j in vals).getValue()
  return res

In [46]:
M=10000
S1= np.array([[0,5,10,2,M,M,M],
              [M,0,4,1,4,M,M],
              [M,M,0,M,3,1,M],
              [M,M,1,0,M,3,M],
              [M,M,M,M,0,M,1],
              [M,M,M,M,M,0,1],
              [M,M,M,M,M,M,0]])
S2=np.array([[0,3,4,6,M,M,M],
              [M,0,2,3,6,M,M],
              [M,M,0,M,1,2,M],
              [M,M,4,0,M,5,M],
              [M,M,M,M,0,M,1],
              [M,M,M,M,M,0,1],
              [M,M,M,M,M,M,0]])
u = (S1,S2)
w=[2,1]
print(solve3(w,u))

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 11 rows, 55 columns and 260 nonzeros
Model fingerprint: 0x296665c7
Variable types: 6 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 40000.000000
Presolve removed 1 rows and 9 columns
Presolve time: 0.00s
Presolved: 10 rows, 46 columns, 195 nonzeros
Variable types: 3 continuous, 43 integer (42 binary)

Root relaxation: objective 3.709091e+01, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   37.09091    0    4 40000.0000   37.09091   100%     -    0s
H    0     0                      43.0000000   37.09091  13.7%  

In [48]:
#4.3
def getw(alpha):
  n=2
  w = [(((n-i-1+1)/n)**alpha) -(((n-i-1)/n)**alpha) for i in range(n)]
  wk = w.copy()
  for i in range(len(wk)-1):
      wk[i] = w[i] -w[i+1]
  return wk
  #return w
print(getw(5))

[0.9375, 0.03125]


In [ ]:
def graph3(nbre_instances):
  N=7
  M=10000
  alphaL=[1,2,3,4,5]
  n=2
  list_de_graph=[]
  for i in range(nbre_instances):
    S1= np.array([[0,random.randint(1,20),random.randint(1,20),random.randint(1,20),M,M,M],
              [M,0,random.randint(1,20),random.randint(1,20),random.randint(1,20),M,M],
              [M,M,0,M,random.randint(1,20),random.randint(1,20),M],
              [M,M,random.randint(1,20),0,M,random.randint(1,20),M],
              [M,M,M,M,0,M,random.randint(1,20)],
              [M,M,M,M,M,0,random.randint(1,20)],
              [M,M,M,M,M,M,0]])
    S2 = np.array([[0,random.randint(1,20),random.randint(1,20),random.randint(1,20),M,M,M],
              [M,0,random.randint(1,20),random.randint(1,20),random.randint(1,20),M,M],
              [M,M,0,M,random.randint(1,20),random.randint(1,20),M],
              [M,M,random.randint(1,20),0,M,random.randint(1,20),M],
              [M,M,M,M,0,M,random.randint(1,20)],
              [M,M,M,M,M,0,random.randint(1,20)],
              [M,M,M,M,M,M,0]])
    list_de_graph.append((S1,S2))
  #print(list_de_graph)
  listeX=[]
  listeY=[]
  for alpha in alphaL:
    w=getw(alpha)
    Xa=[]
    Ya=[]
    for i in range(nbre_instances):
      u=list_de_graph[i]
      dic=solve3(w,u)

      Xa.append(dic[0])
      Ya.append(dic[1])
    listeX.append(Xa)
    listeY.append(Ya)
  print(f"X={listeX} \n Y= {listeY}")
  
  col=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
  for i in range(len(listeX)):
        titre1= "l'impact de la pondération w sur la robustesse de la solution proposée"
        plt.title(titre1)
        plt.xlabel("T1")
        plt.ylabel("T2")
        titre = "alpha = "+str(alphaL[i])
        plt.scatter(listeX[i],listeY[i],label=titre,c=col[i])
        plt.legend()
        #plt.grid()
        plt.savefig(f"nuagefinal{i}.png")
        plt.show()

#graph3(20)